In [1]:
import torch
import torch.nn as nn

In [2]:
class DenseLayer(nn.Module):
  def __init__(self,num_in_feature,growth_rate):
    super(DenseLayer,self).__init__()

    # In the paper it is given that 1x1 will output 4k(k=growth rate) and the final 3x3
    # conv will output only k number of channels
    inner_channel=4*growth_rate

    self.bottle_neck=nn.Sequential(
        nn.BatchNorm2d(num_in_feature),
        nn.ReLU(inplace=True),
        nn.Conv2d(num_in_feature,inner_channel,kernel_size=1,bias=False),
        nn.BatchNorm2d(inner_channel),
        nn.ReLU(inplace=True),
        nn.Conv2d(inner_channel,growth_rate,kernel_size=3,padding=1,bias=False)
    )

    def forward(self,x):
      # The input is N x C x N x M so concatening in dimension=1 means adding in the channel since
      # channel(C) is the the dimension 1 in the above matrix, N in dimension 0, etc...
      return torch.cat([x,self.bottle_neck(x)],1)    

In [3]:
class Transition(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(Transition,self).__init__()

    self.downsample=nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels,out_channels,kernel_size=1,bias=False),
        nn.AvgPool2d(kernel_size=2,stride=2)
    )
  def forward(self,x):
    return self.downsample(x)

In [36]:
class DenseNet(nn.Module):
  def __init__(self,nblocks,growth_rate=12,compression=0.5,num_classes=100):
    super(DenseNet,self).__init__()

    # from the paper: before entering the first denseblock, a convoltuion with 16( or twice
    # the growth rate for DenseNet-BC) output channels is performed on the input image
    inner_channel= 2 * growth_rate
    input_channel=3
    self.growth_rate=growth_rate

    # Since this program is for CIFAR so the first conv is not 7x7 kernel and there is no Maxpool2d
    # 
    self.conv1=nn.Conv2d(input_channel,inner_channel,kernel_size=3,padding=1,bias=False)
    self.features=nn.Sequential()

    # The range is one less coz the last denselayer do not have a transation layer so the last
    # densenet will not be inside this loop.
    for index in range(len(nblocks)-1):
      self.features.add_module('denseblock%d'%(index+1),self._make_dense_layer(inner_channel,nblocks[index]))
      inner_channel+=self.growth_rate*nblocks[index]

      # If a densenet contains m feature-maps, we let the following transition layer generate
      # θm output feature map where 0 < θ ≤ 1 is referred to as the compression factor
      out_channel=int(compression*inner_channel)
      self.features.add_module('transition%d'%(index+1),Transition(inner_channel,out_channel))
      inner_channel=out_channel

    # USING ADD_MODULE BEWARE THAT IF TWO MODULE HAVE THE SAME NAME IT WILL NOT WORK, IT WILL REPLACE
    #          OLD MODULE WITH THE NEW MODULE AND THE NEW MODULE WILL ALSO NOT GET ADDED
    # change the below into len(nblocks)-1 and thsi module will not get added. 
    self.features.add_module('denseblock%d'%(len(nblocks)),self._make_dense_layer(inner_channel,nblocks[len(nblocks)-1]))
    inner_channel+=self.growth_rate*nblocks[len(nblocks)-1]
    self.features.add_module('bn',nn.BatchNorm2d(inner_channel))
    self.features.add_module('relu',nn.ReLU(inplace=True))

    self.avgpool=nn.AdaptiveAvgPool2d((1,1)) # in ImageNet data it is 7x7
    self.classifier=nn.Linear(inner_channel,num_classes)

  def _make_dense_layer(self,in_channel,nblocks):
    dense_block=nn.Sequential()
    for index in range(nblocks):
      dense_block.add_module('bottleneck%d'%index,DenseLayer(in_channel,self.growth_rate))
      in_channel+=self.growth_rate
    return dense_block

  def forward(self,x):
    out=self.conv1(x)
    out=self.features(out)
    out.avgpool(out)
    out=out.view(out.size()[0],-1)
    out=self.classifier(out)
    return out

In [49]:
def densenet121():
  return DenseNet(nblocks=[6,12,24,16],growth_rate=32)

def densenet169():
    return DenseNet(Bottleneck, [6,12,32,32], growth_rate=32)

def densenet201():
    return DenseNet(Bottleneck, [6,12,48,32], growth_rate=32)

def densenet161():
    return DenseNet(Bottleneck, [6,12,36,24], growth_rate=48)

In [38]:
model=densenet121()

In [ ]:
model